In [4]:
from dotenv import load_dotenv
from requests import post, get
import os
import base64
import json
import pandas as pd

load_dotenv()

#getting client_id and secret from environment variables file
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [5]:
def get_token():
    
    #Visit https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow for more information
    
    #base64 encoding
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
    
    url = 'https://accounts.spotify.com/api/token'
    
    headers = {'Authorization': 'Basic ' + auth_base64, 'Content-Type': 'application/x-www-form-urlencoded'}
    
    data = {'grant_type': 'client_credentials'}
    
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {'Authorization': 'Bearer ' + token}

Function to get all songs from a playlist, returns a dictionary with track name and id

In [12]:
def get_playlist(playlist_id, limit=100, fields='items(track(id,name,external_urls))', offset=0, display=True):
    
    headers = get_auth_header(get_token())
    
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields={fields}&limit={limit}&offset={offset}'
    
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    track_info = json_result['items']
    
    track_id_list = []
    track_name_list = []
    tracks={}
    
    for i in range(len(track_info)):
        
        if(display):
            print("\n", i+1, " . ", track_info[i]['track']['name'], " | ", end=" ")
            print(track_info[i]['track']['id'], " | ", end=" ")
            print(track_info[i]['track']['external_urls']['spotify'])
        
        track_id_list.append(track_info[i]['track']['id'])
        track_name_list.append(track_info[i]['track']['name'])

    tracks = dict(zip(track_name_list, track_id_list))
    return tracks

    
# Angry Playlist - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB    
tracks = get_playlist(playlist_id="0KPEhXA3O9jHFtpd1Ix5OB", display=False)  
print(tracks)

{'Shit Luck': '3b1UyW9VxSymsahmyL3qCD', 'Smells Like Teen Spirit': '5ghIJDpPoe3CfHMGu71E6T', 'You Know How I Do': '2OOM9QIzL3xwS0ODsjxUV4', 'Basket Case': '6L89mwZXSOwYl76YXfX13s', 'The Futile': '6Ng8VpVdgIVmJw4HY4eId9', 'Sail': '5xhQChGGhKLWqBqX4XhtYE', 'Break Stuff': '2YC6ET3q1F29B0V7UcPV70', 'I Hate Everything About You': '6rUp7v3l8yC4TKxAAR5Bmx', 'Worst Behavior': '6oF3Es1YzzmLKjGBfThUvD', 'Seventy Times 7': '78HsNTKVNADOaGxS41OSW7', "I've Had It": '5b0kcwMjAES1SKEHBxQHAJ', "I've Been Tired": '2RYe8B8dEfsqzwQYFaKNfD', 'You Oughta Know': '17ZAZ24Eyh5fKqQ06u4R3d', 'Look Back in Anger - 1999 Remaster': '5HKi6OsSg3qngxfuZeLmyX', 'Last Resort': '5eek2X5459T1HoYJk2CKXv', 'Hit Me With Your Best Shot - 1999 Digital Remaster': '7nrtHjt4Vk4Z5jMHOwaS8F', "I'm Shipping Up To Boston": '6Wze1mFZVcT3fj8hTKnBSz', 'Black Skinhead': '722tgOgdIbNe3BEyLnejw4', 'I Fought The Law': '2LWJJNFtrg6PytOdAGrxQB', 'Take Me Out': '6ooluO7DiEhI1zmK94nRCM', 'Revolution - Remastered 2009': '3UDmHZcBTQp8Iu8droNtUl'

Function to get recomendations

In [20]:
def get_recomendations(limit = 20, market = 'IN', seed_artists = None,
  seed_genres = 'classical, country', seed_tracks = None, max_acousticness = None,
  max_danceability = None, max_energy = None,
  max_instrumentalness = None, max_key = None, max_liveness = None,
  max_loudness = None, max_mode = None, max_popularity = None,
  max_speechiness = None, max_tempo = None,
  max_time_signature = None, max_valence = None,
  min_acousticness = None, min_danceability = None,
   min_energy = None,
  min_instrumentalness = None, min_key = None, min_liveness = None,
  min_loudness = None, min_mode = None, min_popularity = None,
  min_speechiness = None, min_tempo = None,
   min_valence = None,
  target_acousticness = None, target_danceability = None,
  target_energy = None, target_loudness = None,
  target_instrumentalness = None, target_key = None,
  target_liveness = None, target_mode = None,
  target_popularity = None, target_speechiness = None,
  target_tempo = None,
  target_valence = None, token = get_token()):
    
    headers = get_auth_header(token)
    
    url = 'https://api.spotify.com/v1/recommendations' 
    
      
    query = f'?limit={limit}&market=IN'\
    f'{f"&target_valence={target_valence}" if target_valence else ""}'\
    f'{f"&target_loudness={target_loudness}" if target_loudness else ""}'\
    f'{f"&target_instrumentalness={target_instrumentalness}" if target_instrumentalness else ""}'\
    f'{f"&target_speechiness={target_speechiness}" if target_speechiness else ""}'\
    f'{f"&target_acousticness={target_acousticness}" if target_acousticness else ""}'\
    f'{f"&target_tempo={target_tempo}" if target_tempo else ""}'\
    f'{f"&target_liveness={target_liveness}" if target_liveness else ""}'\
    f'{f"&target_energy={target_energy}" if target_energy else ""}'\
    f'{f"&target_popularity={target_popularity}" if target_popularity else ""}'\
    f'{f"&target_danceability={target_danceability}" if target_danceability else ""}'\
    f'{f"&target_mode={target_mode}" if target_mode else ""}'\
    f'{f"&target_key={target_key}" if target_key else ""}'\
    f'{f"&seed_artists={seed_artists}" if seed_artists else ""}'\
    f'{f"&seed_genres={seed_genres}" if seed_genres else ""}'\
    f'{f"&seed_tracks={seed_tracks}" if seed_tracks else ""}'
    
    print(query)
    
    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    print (result)
    track_info = json_result['tracks']

    for i in range(len(track_info)):
        print("\n", i+1, " . ", track_info[i]['name'], " | ", end=" ")
        print(track_info[i]['artists'][0]['name'], end=" ")
        print(track_info[i]['external_urls']['spotify'])

Function to get details about the tracks, returns dataframe

In [10]:
def get_song_details(playlist_id):
    
    headers = get_auth_header(get_token())

    tracks = get_playlist(playlist_id=playlist_id, display=False)
    
    ids = list(tracks.values())
    
    ids = str(ids)[1:-1].replace("'","").replace(" ","")
    
    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'
    
    
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    
    df = pd.DataFrame.from_dict(json_result['audio_features'])
    return df

Angry Playlist data - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB 

In [29]:
# Angry Playlist - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB    
angry_df = get_song_details('0KPEhXA3O9jHFtpd1Ix5OB')
angry_df.describe()
#angry_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,0.504824,0.832676,5.323529,-5.945529,0.676471,0.110179,0.067805,0.037413,0.268194,0.500824,120.343382,207971.617647,3.941176
std,0.176102,0.113312,3.327758,2.582891,0.474858,0.090551,0.126797,0.149430,0.189280,0.176570,32.323499,58194.581451,0.238833
min,0.179000,0.435000,0.000000,-12.089000,0.000000,0.029600,0.000025,0.000000,0.045100,0.237000,77.987000,85867.000000,3.000000
25%,0.374500,0.781750,3.000000,-6.764750,0.000000,0.053925,0.001172,0.000000,0.136750,0.351000,93.649500,173646.500000,4.000000
50%,0.500000,0.858500,4.000000,-5.584500,1.000000,0.068750,0.004170,0.000008,0.242500,0.463000,113.352500,208953.000000,4.000000
75%,0.598500,0.909500,8.000000,-4.123000,1.000000,0.156750,0.082925,0.000273,0.333000,0.638750,129.938500,246847.000000,4.000000
max,0.825000,0.997000,11.000000,-1.629000,1.000000,0.340000,0.557000,0.643000,0.868000,0.954000,201.830000,331573.000000,4.000000


Angry Songs Recomendations

In [33]:
#FYI seed_genre matters a lot

get_recomendations(seed_genres="rock", limit=10, target_popularity=100, target_danceability=0.5, target_energy=0.8, target_loudness=-5.94, target_acousticness=0.06, target_valence=0.5, target_tempo=120.34)


?limit=10&market=IN&target_valence=0.5&target_loudness=-5.94&target_acousticness=0.06&target_tempo=120.34&target_energy=0.8&target_popularity=100&target_danceability=0.5&seed_genres=rock
<Response [200]>

 1  .  Time is Running Out  |  Muse https://open.spotify.com/track/2takcwOaAZWiXQijPHIx7B

 2  .  On Melancholy Hill  |  Gorillaz https://open.spotify.com/track/0q6LuUqGLUiCPP1cbdwFs3

 3  .  All Along the Watchtower  |  Jimi Hendrix https://open.spotify.com/track/2aoo2jlRnM3A0NyLQqMN2f

 4  .  In the End  |  Linkin Park https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq

 5  .  broken  |  lovelytheband https://open.spotify.com/track/6XcfKZvJio9Z0fQy11GnNX

 6  .  Born in the U.S.A.  |  Bruce Springsteen https://open.spotify.com/track/0dOg1ySSI7NkpAe89Zo0b9

 7  .  Counting Stars  |  OneRepublic https://open.spotify.com/track/2tpWsVSb9UEmDRxAl1zhX1

 8  .  Round and Round  |  Ratt https://open.spotify.com/track/5B6Z7NwVqA7W3UFiqBdEJC

 9  .  In Too Deep  |  Sum 41 https://open.spot